In [1]:
%load_ext autoreload
%autoreload 2

loading file
[Source](http://files.fast.ai/models/wt103/)

In [2]:
import os
import pickle
import datetime
import pandas as pd
import torch
import torch.nn as nn
import sys

sys.path.append('../')
from training_utils import training_loop, test_loop
from model import RNNLM
# from data_utils import IndexVectorizer, TextDataset, simple_tokenizer

## Downloading pre-trained weights

In [3]:
directory_pre_trained_models = '../../data/weights_pretrained/'
new_data_directory = '../../data/imdb/models/'
os.makedirs(directory_pre_trained_models, exist_ok=True)

In [5]:
# !wget http://files.fast.ai/models/wt103/fwd_wt103_enc.h5 -O $directory_pre_trained_models/fwd_wt103_enc.h5

Can skip this one V

In [6]:
# !wget http://files.fast.ai/models/wt103/fwd_wt103.h5 -O $directory_pre_trained_models/fwd_wt103.h5

In [7]:
# !wget http://files.fast.ai/models/wt103/itos_wt103.pkl -O $directory_pre_trained_models/fitos_wt103.pkl

In [4]:
!ls -ltha $directory_pre_trained_models

total 886M
drwxrwxr-x. 5 frido frido   56 Feb 12 21:38 ..
-rwxr-xr-x. 1 root  root  441M Feb 12 17:18 fwd_wt103.h5
drwxrwsr-x. 2 root  root    73 Feb 12 17:18 .
-rwxr-xr-x. 1 root  root  441M Feb 12 17:18 fwd_wt103_enc.h5
-rwxr-xr-x. 1 root  root  4.0M Feb 12 17:18 fitos_wt103.pkl


## Loading weights

In [4]:
encoder_file = os.path.join(directory_pre_trained_models, 'fwd_wt103.h5')
weights_file = os.path.join(directory_pre_trained_models, 'fwd_wt103_enc.h5')
fitos_file = os.path.join(directory_pre_trained_models, 'fitos_wt103.pkl')
vectorizer_file = os.path.join(new_data_directory, 'lm_vectorizer.pkl')

In [5]:
wgts = torch.load(weights_file, map_location=lambda storage, loc: storage)

In [6]:
enc = torch.load(encoder_file, map_location=lambda storage, loc: storage)

In [7]:
# GPU variables
use_gpu = torch.cuda.is_available()
device_num = 0
device = torch.device(f"cuda:{device_num}" if False else "cpu")
device

device(type='cpu')

In [8]:
vocab_size = 238462
embedding_size = 400
hidden_size = 1150
num_layers = 3

In [9]:
# # load weights
# for k, v in enc.items():    
#     layer_detail = k.split('.')
#     layer_name = layer_detail[-1].replace('_raw', '')
    
#     print(layer_detail)

#     try:
#         layer = getattr(lm, layer_detail[1])
#     except Exception as e:
#         print(e)
        
#     # which rnn, and what layer???
#     if layer_detail[1] == 'rnns':
#         n_rnn = int(layer_detail[2])
#         layer = layer[n_rnn]
#     try:
#         # this is what assigns the new value
#         getattr(layer, layer_name).data = v
    
#     except Exception as e:
#         print(e)(layer_detail)

['0', 'encoder', 'weight']
name 'lm' is not defined
name 'layer' is not defined


TypeError: 'NoneType' object is not callable

In [10]:
new_enc = {}
for k,v in enc.items():
    layer_detail = k.split('.')
    layer_name = layer_detail[-1].replace('_raw', '')
    if len(layer_detail) == num_layers: 
        new_enc[f'{layer_detail[1]}.{layer_name}'] = v
    else:
        new_enc[f'{layer_detail[1]}.{layer_detail[2]}.{layer_name}'] = v
    

In [11]:
# Remove this odd element as it is the same as encoder.weight
#new_enc['encoder_with_dropout.embed.weight'] == new_enc['encoder.weight']
del new_enc['encoder_with_dropout.embed.weight']

In [12]:
# Load our vectorizer

## Load the wikitext vocabulary
pretrained_idx2word = pickle.load(open(fitos_file, 'rb'))

In [13]:
pretrained_word2idx = {k: i for i,k in enumerate(pretrained_idx2word)}

In [14]:
new_model_vectorizer = pickle.load(open(vectorizer_file, 'rb'))

In [15]:
pretrained_encoder_weights = enc['0.encoder.weight']

In [16]:
row_m = pretrained_encoder_weights.mean(dim=0)

In [17]:
row_m = [x.item() for x in row_m]

In [18]:
new_vocab_size = len(new_model_vectorizer.word2idx)
new_encoder_weights = torch.tensor([row_m for i in range(new_vocab_size)])

In [19]:
new_idx2weights = {}
for word, i in new_model_vectorizer.word2idx.items():
    if word in pretrained_word2idx:
        word_idx = pretrained_word2idx[word]
        new_encoder_weights[i] = pretrained_encoder_weights[word_idx]

In [48]:
import copy
new_enc['encoder.weight'] = new_encoder_weights
new_enc['decoder.weight'] = copy.copy(new_encoder_weights)
new_enc['decoder.bias'] = torch.zeros(new_enc['decoder.weight'].shape[0])

In [26]:
for k,v in lm.state_dict().items():
    print(k, v.shape)

encoder.weight torch.Size([20004, 400])
rnns.0.weight_ih_l0 torch.Size([4600, 400])
rnns.0.weight_hh_l0 torch.Size([4600, 1150])
rnns.0.bias_ih_l0 torch.Size([4600])
rnns.0.bias_hh_l0 torch.Size([4600])
rnns.1.weight_ih_l0 torch.Size([4600, 1150])
rnns.1.weight_hh_l0 torch.Size([4600, 1150])
rnns.1.bias_ih_l0 torch.Size([4600])
rnns.1.bias_hh_l0 torch.Size([4600])
rnns.2.weight_ih_l0 torch.Size([1600, 1150])
rnns.2.weight_hh_l0 torch.Size([1600, 400])
rnns.2.bias_ih_l0 torch.Size([1600])
rnns.2.bias_hh_l0 torch.Size([1600])
decoder.weight torch.Size([20004, 400])
decoder.bias torch.Size([20004])


In [49]:
# example of a model
lm = RNNLM(device=device, vocab_size=new_vocab_size, 
           embedding_size=embedding_size, hidden_size=hidden_size, 
           batch_size=64, num_layers=3, tie_weights=True)

In [50]:
id(lm.encoder.weight) == id(lm.decoder.weight)

True

In [51]:
lm.load_state_dict(new_enc)

In [52]:
id(lm.encoder.weight) == id(lm.decoder.weight)

True

In [9]:
?RNNLM

In [36]:
a = [1,2,3]
id(a)

139644474457288

In [37]:
b = a

In [38]:
id(b)

139644474457288

In [31]:
a = [4,5,6]

In [33]:
b = [7,8,9]

In [34]:
b

[7, 8, 9]

In [35]:
id(a)

139644473930248